In [1]:
import pandas as pd
import numpy as np
import os
import pyarrow as pa
import pyarrow.parquet as pq

### 因子构造样例

#### 预先读取merge_daily_info

In [2]:
merge_daily_info = pd.read_parquet(r'/Users/xuyanye/Desktop/quant mm/factor/data/merge_daily_info.parquet')
merge_daily_info['TradingDate'] = pd.to_datetime(merge_daily_info['TradingDate'])
merge_daily_info = merge_daily_info.drop_duplicates(subset=['Stkcd','TradingDate'])
TRD_Dalyr = pd.read_parquet(r'/Users/xuyanye/Desktop/quant mm/factor/data/TRD_Dalyr.parquet')

In [3]:
TRD_Dalyr.columns

Index(['Stkcd', 'TradingDate', 'Opnprc', 'Hiprc', 'Loprc', 'Clsprc',
       'Dnshrtrd', 'Dnvaltrd', 'Dsmvosd', 'Dsmvtll', 'Dretwd', 'Dretnd',
       'Adjprcwd', 'Adjprcnd', 'Markettype', 'Capchgdt', 'Trdsta',
       'Ahshrtrd_D', 'Ahvaltrd_D', 'PreClosePrice', 'ChangeRatio', 'LimitDown',
       'LimitUp', 'LimitStatus'],
      dtype='object')

In [4]:
#数据提取与合并
Opnprc = TRD_Dalyr[['TradingDate','Stkcd','Opnprc']]
Clsprc = TRD_Dalyr[['TradingDate','Stkcd','Clsprc']]
calc_cvturn = pd.merge(Opnprc, Clsprc, on=['TradingDate', 'Stkcd'], how='left')
# 现在 calc_factor_data 应该只包含 'TradingDate', 'Stkcd', 'Opnprc', 'Clsprc'
calc_cvturn.columns

Index(['TradingDate', 'Stkcd', 'Opnprc', 'Clsprc'], dtype='object')

# alpha008

In [5]:
COLUMN_FACTOR_NAME = "alpha008" 
ALPHA_SAVE_PATH_DIR = "/Users/xuyanye/Desktop/quant mm/factor/alpha"
if not os.path.exists(ALPHA_SAVE_PATH_DIR):
    os.makedirs(ALPHA_SAVE_PATH_DIR)
# 定义最终因子在DataFrame中的列名，以及Parquet文件的基础名称
# 这两部分必须一致，才能被您的回测代码正确识别

In [6]:
# --- 因子计算步骤 ---
calc_factor_data = calc_cvturn.copy()

# 1. 确保数据按股票代码和交易日期排序
# 这一步对于计算时间序列相关的函数（如 diff, rolling, shift）至关重要
calc_factor_data = calc_factor_data.sort_values(by=['Stkcd', 'TradingDate']).copy()

# 2. 计算 daily returns (日收益率)
# 基于 Clsprc 列计算
calc_factor_data['daily_returns'] = calc_factor_data.groupby('Stkcd')['Clsprc'].pct_change()

# 3. 计算 sum(open, 5) - 过去5个交易日的开盘价之和
calc_factor_data['sum_open_5'] = calc_factor_data.groupby('Stkcd')['Opnprc'].rolling(window=5, min_periods=1).sum().reset_index(level=0, drop=True)

# 4. 计算 sum(returns, 5) - 过去5个交易日的日收益率之和
calc_factor_data['sum_returns_5'] = calc_factor_data.groupby('Stkcd')['daily_returns'].rolling(window=5, min_periods=1).sum().reset_index(level=0, drop=True)

# 5. 计算 product_term = (sum(open,5) * sum(returns,5))
calc_factor_data['product_term'] = calc_factor_data['sum_open_5'] * calc_factor_data['sum_returns_5']

# 6. 计算 delay(product_term, 10) - product_term 10个交易日之前的值
calc_factor_data['delay_product_term_10'] = calc_factor_data.groupby('Stkcd')['product_term'].shift(10)

# 7. 计算 inner_diff = product_term - delay(product_term, 10)
calc_factor_data['inner_diff_term'] = calc_factor_data['product_term'] - calc_factor_data['delay_product_term_10']

# 8. 计算 rank(inner_diff) - 对 inner_diff_term 进行跨截面（每天）排名
# 'method="average"' 是 pandas 默认的排名方式，'ascending=True' 默认是升序排名
calc_factor_data['ranked_inner_diff'] = calc_factor_data.groupby('TradingDate')['inner_diff_term'].rank(method='average', ascending=True)

# 9. 对 ranked_inner_diff 进行标准化 (z-score 标准化)
calc_factor_data['ranked_inner_diff_standardized'] = calc_factor_data.groupby('TradingDate')['ranked_inner_diff'].transform(lambda x: (x - x.mean()) / x.std())

# 10. 计算最终的因子值 (这里我们称之为 alpha_new_factor)
# factor = -1 * ranked_inner_diff_standardized
calc_factor_data['alpha_new_factor'] = -1 * calc_factor_data['ranked_inner_diff_standardized']


In [7]:
# 假设 merge_daily_info 是另一个包含 Stkcd 和 TradingDate 的 DataFrame
# 将计算出的 Alpha08_Factor 合并到 merge_daily_info
# 注意：如果 calc_cvturn 就是最终要用的 DataFrame，可能不需要这一步合并
# 这里为了与用户提供的格式保持一致，假设有一个目标 DataFrame 'merge_daily_info'
# 如果 calc_cvturn 本身就是最终结果，则可以跳过此合并步骤
# 请根据实际情况调整 'merge_daily_info' 的定义
try:
    # 模拟 merge_daily_info 如果它不存在
    if 'merge_daily_info' not in locals():
        print("警告：'merge_daily_info' 未定义。为演示目的，将假设 merge_daily_info 等同于 calc_cvturn 的基础部分。")
        merge_daily_info = calc_cvturn[['Stkcd', 'TradingDate']].copy() # 只是一个示例

    # 进行合并
    calc_cvturn = pd.merge(merge_daily_info,
                           calc_factor_data[['Stkcd','TradingDate','alpha_new_factor']],
                           on=['Stkcd','TradingDate'],
                           how='left')
    calc_cvturn = calc_cvturn.rename(columns={'alpha_new_factor': 'alpha008'})
except NameError:
    print("错误：'merge_daily_info' 未定义，请确保该DataFrame存在后再执行合并。")


# 去掉缺失值与inf (根据用户提供的格式)
# 这一步会移除 Alpha12_Factor 列中因为 delta 计算导致的 NaN 值 (例如每个股票的第一个交易日)
calc_cvturn = calc_cvturn.dropna(subset=['alpha008']) # 只针对因子列去除NaN，避免影响其他数据
calc_cvturn = calc_cvturn[~np.isinf(calc_cvturn['alpha008'])] # 确保因子值不是无穷大

# 打印结果 DataFrame 的头部，以供检查
print(calc_cvturn.head())

    Stkcd TradingDate  alpha008
11      1  1991-04-18 -0.000000
12      1  1991-04-19  0.612372
14      1  1991-04-23  0.365148
15      1  1991-04-24  0.612372
16      1  1991-04-25  1.095445


In [9]:
##采用parquet优化内存
save_path = r'/Users/xuyanye/Desktop/quant mm/factor/alpha/alpha008.parquet'
calc_cvturn.to_parquet(save_path, compression='gzip')